In [ ]:
"""
Spot counts for cells with highly expressed genes (dense spots)
1. Measure total intensity of every ROI after bleed-through correction and background subtraction.
2. Calculate the number of spot from total intensity based on unitary spot fluorescence intensity
3. Correlate the number of spots (from air-localize) with the total fluorescence intensity/voxel in each ROI and determine a 'cutoff'. 
   Spot count > cutoff: use spot count converted based on total fluorescence intensity; 
   Spot count < cutoff: use spot count from Airlocalize
"""

import os, sys,z5py
import numpy as np
import pandas as pd
from glob import glob 
from skimage.measure import regionprops
from skimage.io import imread, imsave
from os.path import abspath, dirname
from scipy import stats
from scipy.stats import skewnorm,lognorm
from scipy.optimize import minimize
import itertools

In [ ]:
### Specify directories to data files
input_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs"
out_dir = os.path.join(input_dir, "testfull-aug5-v2")  
roi_csv = os.path.join(out_dir, 'roi.csv')
f_spots = os.path.join(out_dir, f'spots.csv')
out_csv = os.path.join(out_dir, 'spotcount_dense_spot_corrected.csv')

# for every gene
rounds = ['r1', 'r2']
channels = ['c0', 'c1', 'c2', 'c4']

# r1 should be the wrappped one
fx_spots = [os.path.join(input_dir, f'spots_pooled\\spots_{r}_{c}.txt') 
                 for r, c in itertools.product(rounds, channels)]
fx_intns = [os.path.join(input_dir, f'intensities_pooled\\{r}_{c}_intensity.csv') 
                 for r, c in itertools.product(rounds, channels)]

for f in fx_spots:
    assert os.path.isfile(f)
for f in fx_intns:
    assert os.path.isfile(f)
assert os.path.isfile(f_spots)

In [ ]:
roi = pd.read_csv(roi_csv, sep=',', index_col=0)# .set_index('roi')
roi

In [ ]:
spotcount = pd.read_csv(f_spots, sep=',', index_col=0)
spotcount.columns = [col.replace('spots_', '') for col in spotcount.columns]
spotcount

In [ ]:
### Identify unit spot intensity for every gene
unit_intns = []
for i, (r, c) in enumerate(itertools.product(rounds, channels)):
    f = fx_spots[i]
    print(r, c, f)
    
    # (x, y, z, I)
    spot=np.loadtxt(f, delimiter=',')
    vox=[0.92,0.92,0.84]
    spot[:,:3]=spot[:,:3]/vox  # convert from physical unit to pixel unit
    ##remove spots on edges (eliminate false detection)
    spot = spot[np.logical_and(spot[:,0]<=1500, spot[:,0]>250)]
    spot = spot[np.logical_and(spot[:,1]<=1500, spot[:,1]>250)]
    spot = spot[np.logical_and(spot[:,2]<=650,  spot[:,2]>150)]   
    
    ## assign the most frequent intensity as the single-spot-intensity
    spot_int = spot[:,3]
    spot_int = spot_int[spot_int!=-8.0] # ???
    n,b=np.histogram(spot_int, bins=5000)
    unit_intn = b[np.argmax(n)]
    unit_intns.append(unit_intn)
    
unit_intns

In [ ]:
# df_count is spot count calculated from total fluorescence intensity
df_count = pd.DataFrame(index=roi.index, dtype=float)
for i, (r, c) in enumerate(itertools.product(rounds, channels)):
    f = fx_intns[i]
    unit_int = unit_intns[i]
    print(r, c, f)

    cell_int = pd.read_csv(f, sep=',', index_col=0)
    cell_int = cell_int.reindex(roi.index) ## only include intact ROIs###

    vec_mean = cell_int['mean_intensity'].values
    vec_area = roi['area'].values

    # background
    n,b = np.histogram(vec_mean, bins=1000)
    bg = b[np.argmax(n)]    
    
    # count
    vec_count = np.clip(vec_mean - bg, 0, None)*vec_area/unit_int
    df_count[f'{r}_{c}'] = vec_count

df_count

In [ ]:
# proc and save
# update spotcount using df_count
df_cutoff = spotcount.copy()
# loop over genes
for i in df_count.columns:
    density = spotcount[i]/(roi['area']*2*2*2/(0.92*0.92*0.84)) # convert um^3 to voxel values
    cond = density > 0.01 ##this threshold corresponds to spot-spot distance ~1.3 um apart
    print(i, cond.sum())
    df_cutoff.loc[cond, i] = df_count.loc[cond, i].copy()
    
df_cutoff

In [ ]:
df_cutoff.to_csv(out_csv)

In [ ]:
test = pd.read_csv(out_csv, index_col=0)
for col in test:
    print(np.max(test[col]))
test